## 머신러닝

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트 데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 미용
- 프로세스 : 텍스트 전처리 > 특성 벡터화 > 머신러닝 모델 구축, 학습, 평가
- 텍스트 전처리 : 토큰화, 불용어 제거, 표제어 추출, 형태소 분석...

##### 특성 벡터화/추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
- BowW, Word2ve 라이브러리(모듈)존재

##### LDA
- 문서에 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLSAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_test.txt, ratings_train.txt 다운로드

- id(리뷰번호), document(리뷰), label(감성분류 클래스 0은 부정, 1은 긍정 감성)

In [62]:
## 필요 라이브러리 사용등록
import pandas as pd
import re

In [63]:
# 모든 모듈/라이브러리는 __version__
pd.__version__

'2.2.1'

In [64]:
## pandas 버전 상세정보
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : bdc79c146c2e32f2cab629be240f01658cfb6cc2
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Windows
OS-release            : 11
Version               : 10.0.22000
machine               : AMD64
processor             : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : Korean_Korea.949

pandas                : 2.2.1
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.8.2
setuptools            : 69.1.1
pip                   : 24.0
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : None
html5lib              : None
pymysql               : None
psycopg2              : None

In [65]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [66]:
dfNsmcTrain

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [67]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [68]:
dfNsmcTrain = dfNsmcTrain[ dfNsmcTrain['document'].notnull()]

In [69]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [70]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [71]:
## Ragular Expression(정규식)으로 한글 이외의 문제는 다 제거
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ|가-힣]+', ' ', x))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_848\2606043373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ|가-힣]+', ' ', x))


In [72]:
dfNsmcTrain.head()


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [73]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [74]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [75]:
dfNsmcTest.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [76]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [77]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [78]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [79]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [80]:
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ|가-힣]+', ' ', x))

In [81]:
dfNsmcTest

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [82]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터를 제거
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'] != ' ']

In [83]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149186 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149186 non-null  int64 
 1   document  149186 non-null  object
 2   label     149186 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [84]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'] != ' ']

In [85]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49726 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49726 non-null  int64 
 1   document  49726 non-null  object
 2   label     49726 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키지의 Okt 클래스 사용

##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템 등록정보 JAVA-HOME 설정
    - 프로그램 실행 sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템변수 > 새로만들기 버튼 클릭
    - JAVA_HOME과 경로 추가등록
- JPype1 : 파이썬에서 JDK를 사용하게 해주는 프로그램
- koNLPy 설치

In [86]:
# JPype1 설치
!pip install JPype1

   ---------------------------------------- 0.0/352.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/352.2 kB ? eta -:--:--
   ------ -------------------------------- 61.4/352.2 kB 825.8 kB/s eta 0:00:01
   ---------------------------------------- 352.2/352.2 kB 3.7 MB/s eta 0:00:00


In [87]:
# koNLPy 설치
!pip install koNLPy

   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/19.4 MB 1.1 MB/s eta 0:00:18
    --------------------------------------- 0.3/19.4 MB 3.3 MB/s eta 0:00:06
   - -------------------------------------- 0.8/19.4 MB 5.7 MB/s eta 0:00:04
   -- ------------------------------------- 1.4/19.4 MB 7.2 MB/s eta 0:00:03
   ---- ----------------------------------- 2.1/19.4 MB 8.9 MB/s eta 0:00:02
   ------ --------------------------------- 3.0/19.4 MB 10.7 MB/s eta 0:00:02
   -------- ------------------------------- 4.1/19.4 MB 12.5 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/19.4 MB 14.7 MB/s eta 0:00:01
   -------------- ------------------------- 7.1/19.4 MB 16.8 MB/s eta 0:00:01
   ---------------- ----------------------- 8.0/19.4 MB 17.0 MB/s eta 0:00:01
   ----------------- ---------------------- 8.7/19.4 MB 16.9 MB/s eta 0:00:01
   ----

In [88]:
import konlpy

In [89]:
konlpy.__version__

'0.6.0'

In [98]:
## 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [91]:
okt = Okt()

In [93]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [96]:
tfidf = TfidfVectorizer(tokenizer=oktTokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)

In [97]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

c:\DEV\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [99]:
nsmc_train_tfidf

<149186x115705 sparse matrix of type '<class 'numpy.float64'>'
	with 2712786 stored elements in Compressed Sparse Row format>

In [100]:
with open('result.npy', 'wb') as f:
    np.save(f, nsmc_train_tfidf)